In [84]:
from pyspark.sql import SparkSession

In [85]:
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'


In [86]:
# spark = SparkSession.builder \
#     .appName("KafkaSparkStreaming") \
#     .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
#     .getOrCreate()

In [87]:
spark = SparkSession.builder.appName("kafkaWordCount")\
        .config("spark.jars.packages",
        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-avro_2.12:3.5.0") \
        .getOrCreate()

In [89]:
kafka_df = spark.readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers","localhost:9092")\
    .option("subscribe","listen")\
    .load()


AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

In [68]:
from pyspark.sql.types import *

In [69]:
schema = StructType([
    StructField("customer_id",IntegerType(),True),
    StructField("transaction_type",StringType(),True),
    StructField("transaction_amount",IntegerType(),True)
])

df = spark.readStream.format("csv").option("header","True").option("header","True").schema(schema).load("file:///home/hdoop/notebooks/data/spark_practice/csvToKafka")

In [70]:
df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- transaction_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)



In [71]:
df2 = df.selectExpr("Cast(customer_id as STRING) as key","to_json(struct(*)) as value")

In [72]:
kafka_df = df2.writeStream.format("kafka").option("kafka.bootstrap.servers","localserver:9092").option("topic","listen").start().awaitTermination()

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

In [83]:
spark.stop()